<a href="https://colab.research.google.com/github/Baldros/NLP-Course-HuggingFace/blob/main/2.4.%20Handling_multiple_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação:

    O objetivo aqui é estudar como agrupar sequências de entrada em lotes
    (batchs). Geralmente, as frases que queremos passar para o nosso modelo
    não terão todas o mesmo comprimento.

    Essa técnica de se trabalhar com lotes (batchs) é muito útil para
    otimização do processo, tendo em vista, que trabalhar a entrada
    unidade por unidade seria bem mais custoso.

# Trabalhando com batchs:

    A ideia aqui é ver como esse processo de tokenização
    funciona, de modo que aqui, difernte do modulo anterior
    que se era dado os valores tokenizados, vamos tokenizar.

In [1]:
# Sequencia (frase) que utilizaremos:
sequence = "I've been waiting for a HuggingFace course my whole life." # Servirá para os dois casos!

**Pytorch**

In [2]:
# Importações:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
# Escolhendo modelo pelo checkpoint:
checkpoint_pt = "distilbert-base-uncased-finetuned-sst-2-english"

# Instanciando tokenizador:
tokenizer_pt = AutoTokenizer.from_pretrained(checkpoint_pt)

# Instanciando modelo:
model_pt = AutoModelForSequenceClassification.from_pretrained(checkpoint_pt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [4]:
# Tokenizando a sequencia:
tokens_pt = tokenizer_pt.tokenize(sequence)

# Covertendo os tokens para ids:
ids_pt = tokenizer_pt.convert_tokens_to_ids(tokens_pt)

# Transformando os ids em tensor:
input_ids_pt = torch.tensor(ids_pt)

# Introduzindo os inputs:
model_pt(input_ids_pt) # Essa linha irá falha!

IndexError: too many indices for tensor of dimension 1

**Tensorflow**

In [5]:
# Importações:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [6]:
# Escolhendo modelo pelo checkpoint:
checkpoint_tf = "distilbert-base-uncased-finetuned-sst-2-english"

# Instanciando o tokenizer:
tokenizer_tf = AutoTokenizer.from_pretrained(checkpoint_tf)

# Instanciando o modelo:
model_tf = TFAutoModelForSequenceClassification.from_pretrained(checkpoint_tf)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [7]:
# Tokenizando a sequencia:
tokens_tf = tokenizer_tf.tokenize(sequence)

# Covertendo os tokens para ids:
ids_tf = tokenizer_tf.convert_tokens_to_ids(tokens_tf)

# Transformando os ids em tensor:
input_ids_tf = tf.constant(ids_tf)

# Jogando as entradas no modelo:
model_tf(input_ids_tf)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.7276218,  2.8789387]], dtype=float32)>, hidden_states=None, attentions=None)

    Isso acontece porque durante a tokenização e associação de cada token
    aos seus IDs de entrada correspondentes, obtemos duas listas de
    comprimentos diferentes. Tentar criar um tensor ou uma matriz NumPy
    a partir dessas duas listas resultará em um erro, pois todos os arrays
    e tensores devem ser retangulares.

    
    O problema é que enviamos uma única sequência para o modelo, enquanto
    os modelos 🤗 Transformers esperam várias frases por padrão. Aqui,
    tentamos fazer tudo o que o tokenizador fez nos bastidores quando o
    aplicamos a uma sequência. Mas se você observar atentamente, verá que
    o tokenizador não apenas converteu a lista de IDs de entrada em um
    tensor, ele adicionou uma dimensão acima dela:

**Pytorch**

In [8]:
# Checando a saída:
tokenized_inputs_pt = tokenizer_pt(sequence, return_tensors="pt")
print(tokenized_inputs_pt["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


**Tensorflow**

In [9]:
tokenized_inputs_tf = tokenizer_tf(sequence, return_tensors="tf")
print(tokenized_inputs_tf["input_ids"])

tf.Tensor(
[[  101  1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026
   2878  2166  1012   102]], shape=(1, 16), dtype=int32)


    Uma maneira de contornar isso é garantir que a segunda frase tenha
    o mesmo comprimento que a primeira, adicionando um token especial
    quantas vezes forem necessárias. Outra maneira seria truncar a primeira
    sequência ao comprimento da segunda, mas isso faria com que perdesse
    muitas informações que podem ser necessárias para classificar
    corretamente a frase. Vamos adicionar outra dimensão.

**Pytorch**

In [10]:
# Importações:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [11]:
# Instanciamento pelo Checkpoint:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer_pt = AutoTokenizer.from_pretrained(checkpoint)
model_pt= AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Transformação dos inputs:
tokens_pt = tokenizer_pt.tokenize(sequence)
ids_pt = tokenizer_pt.convert_tokens_to_ids(tokens_pt)

# Visualização dos inputs:
input_ids_pt = torch.tensor([ids_pt])
print("Input IDs:", input_ids_pt)

# Visualização dos logits:
output_pt = model_pt(input_ids_pt)
print("Logits:", output_pt.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [12]:
# Checando os lotes:
batched_ids_pt = [ids_pt, ids_pt]
print(batched_ids_pt)

[[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012], [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]]


**Tensorflow**

In [13]:
# Importações:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [14]:
# Instanciamento pelo Checkpoint:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer_tf = AutoTokenizer.from_pretrained(checkpoint)
model_tf = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

# Transformação dos inputs:
tokens_tf = tokenizer_tf.tokenize(sequence)
ids_tf = tokenizer_tf.convert_tokens_to_ids(tokens_tf)

# Visualização dos inputs:
input_ids_tf = tf.constant([ids_tf])
print("Input IDs:", input_ids_tf)

# Visualização dos logits:
output_tf = model_tf(input_ids_tf)
print("Logits:", output_tf.logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Input IDs: tf.Tensor(
[[ 1045  1005  2310  2042  3403  2005  1037 17662 12172  2607  2026  2878
   2166  1012]], shape=(1, 14), dtype=int32)
Logits: tf.Tensor([[-2.7276218  2.8789387]], shape=(1, 2), dtype=float32)


In [15]:
# Checando os lotes:
batched_ids_tf = [ids_tf, ids_tf]
print(batched_ids_tf)

[[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012], [1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]]


    O Batching (ou agrupamento) é o ato de enviar várias frases pelo
    modelo, todas de uma vez. Se você tiver apenas uma frase, pode
    simplesmente construir um lote com uma única sequência:

    O agrupamento permite que o modelo funcione quando você o alimenta
    com várias frases. Usar várias sequências é tão simples quanto construir
    um lote com uma única sequência. No entanto, há uma segunda questão.
    Ao tentar agrupar duas (ou mais) frases, elas podem ter comprimentos
    diferentes. Se você já trabalhou com tensores antes, sabe que eles
    precisam ter uma forma retangular, então você não poderá converter
    a lista de IDs de entrada diretamente em um tensor. Para contornar
    |esse problema, geralmente preenchemos as entradas.

# Preenchendo as entradas:

    Para contornar isso, usaremos preenchimento para garantir que
    nossos tensores tenham uma forma retangular. O preenchimento garante
    que todas as nossas frases tenham o mesmo comprimento, adicionando
    uma palavra especial chamada de token de preenchimento às frases com
    menos valores. Por exemplo, se você tiver 10 frases com 10 palavras
    e 1 frase com 20 palavras, o preenchimento garantirá que todas as
    frases tenham 20 palavras. Em nosso exemplo, o tensor resultante se
    parece com isto:







In [16]:
# Quantidade
padding_id = 100

# Padding:
batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

    O ID do token de preenchimento pode ser encontrado em tokenize
    .ad_token_id(Atributo). Vamos usá-lo e enviar nossas duas frases
    através do modelo individualmente e em batchs juntas:

In [17]:
# Definindo as sequencias de tamanhos diferentes:
sequence1_ids = [[200, 200, 200]]
sequence2_ids= [[200, 200]]

    To definindo as sequencias antes de separar entre pytorch
    e tensorflow para separar só o que realmente for diferente
    uma biblioteca da outra mesmo.

**Pytorch**

In [18]:
# Instanciando modelo:
model_pt = AutoModelForSequenceClassification.from_pretrained(checkpoint) # Lembre-se que o checkpoint foi definido lá em cima.

# Completando os lotes de ids para não dar erro:
batched_ids_pt = [
    [200, 200, 200],
    [200, 200, tokenizer_pt.pad_token_id],
]

print(model_pt(torch.tensor(sequence1_ids)).logits)
print(model_pt(torch.tensor(sequence2_ids)).logits)
print(model_pt(torch.tensor(batched_ids_pt)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


**Attention Mask**

    As máscaras de atenção são tensores com a mesma forma que o tensor
    de IDs de entrada, preenchidos com 0s e 1s. Os 1s indicam que os tokens
    correspondentes devem ser considerados, enquanto os 0s indicam que os
    tokens correspondentes devem ser ignorados pelas camadas de atenção do
    modelo. Essa técnica é essencial em modelos de processamento de linguagem
    natural, permitindo a gestão de sequências de comprimentos variáveis,
    consideração contextual eficiente, economia de recursos computacionais,
    tratamento adequado de tokens de preenchimento e redução de overfitting.
    Em resumo, as máscaras de atenção melhoram o desempenho e a eficiência
    dos modelos de NLP.

    Note que funciona, mas a própria biblioteca acusa a necessidade de
    colocar utilizar uma máscara de atenção. Testandoo trecho com uma
    mascada de atenção:

In [21]:
# Máscara de atenção:
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

In [25]:
# Outputs com a máscada de atenção:
outputs_mask_pt = model_pt(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs_mask_pt.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


**Tensorflow**

In [24]:
# Instanciando modelo:
model_tf = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

# Completando batchs:
batched_ids_tf = [
    [200, 200, 200],
    [200, 200, tokenizer_tf.pad_token_id],
]

# Saídas:
print(model_tf(tf.constant(sequence1_ids)).logits)
print(model_tf(tf.constant(sequence2_ids)).logits)
print(model_tf(tf.constant(batched_ids_tf)).logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tf.Tensor([[ 1.5693682 -1.3894583]], shape=(1, 2), dtype=float32)
tf.Tensor([[ 0.5803017  -0.41252568]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[ 1.569368  -1.3894587]
 [ 1.33735   -1.2163203]], shape=(2, 2), dtype=float32)


    Note que no caso do Tensorflow, como em alguns outros casos,
    o cerne da resposta é a mesma, mas sempre vem com alguma
    notificação a mais. Aqui não foi pedido a máscara de atenção,
    de todo modo, podemos utilizar a máscada de atenção aqui também
    sem demais problemas.

In [26]:
# Outputs com a máscada de atenção:
outputs_mask_tf = model_tf(tf.constant(batched_ids), attention_mask=tf.constant(attention_mask))
print(outputs_mask_tf.logits)

tf.Tensor(
[[ 1.569368   -1.3894587 ]
 [ 0.58030075 -0.4125247 ]], shape=(2, 2), dtype=float32)


    Com modelos Transformer, existe um limite para o comprimento das
    sequências que podemos passar aos modelos. A maioria dos modelos
    lida com sequências de até 512 ou 1024 tokens e travará ao ser
    solicitada a processar sequências mais longas. Existem duas
    soluções para esse problema:
  
    Use um modelo com um comprimento de sequência suportado mais longo.
    Trunque suas sequências. Os modelos têm diferentes comprimentos de
    sequência suportados, e alguns são especializados em lidar com
    sequências muito longas. O Longformer é um exemplo, e outro é o LED.
    Se você está trabalhando em uma tarefa que requer sequências muito
    longas, recomendamos que você dê uma olhada nesses modelos.

Longformer: https://huggingface.co/docs/transformers/model_doc/longformer

LED: https://huggingface.co/docs/transformers/v4.39.0/en/model_doc/led

    Caso contrário, recomendamos que você trunque suas sequências
    especificando o parâmetro max_sequence_length:

